# Step 3 - Run the Model

Now that we have finished training the model, we can use it to drive the car. Start the AirSim exectuable in a different window, and change the MODEL_FILENAME parameter to point to your downloaded weights. We have included a sample model in case you need it (please note that this is not a perfectly trained model and is only being provided to you as a reference).

In [1]:
%matplotlib inline
%matplotlib notebook
from Share.scripts_downpour.app.airsim_client import *
from Share.scripts_downpour.app.rl_model import RlModel
import matplotlib.pyplot as plt
import os
import numpy as np
import time
import sys
import json
import PIL
import PIL.ImageFilter
import datetime
from IPython.display import clear_output

airsim_path = 'E:\\AD_Cookbook_AirSim\\'
data_dir = os.path.join(os.getcwd(), 'Share')

MODEL_FILENAME = 'Share\\checkpoint\\BIBIT\\Tahap 2\\13219.json' #Your model goes here

C:\Miniconda\envs\Alpaca\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Miniconda\envs\Alpaca\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Miniconda\envs\Alpaca\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Miniconda\envs\Alpaca\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passi

First, we load the model from disk

In [2]:
model = RlModel(None, False)
with open(MODEL_FILENAME, 'r') as f:
    checkpoint_data = json.loads(f.read())
    model.from_packet(checkpoint_data['model'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 59, 255, 3)        0         
_________________________________________________________________
convolution0 (Conv2D)        (None, 59, 255, 16)       448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 29, 127, 16)       0         
_________________________________________________________________
convolution1 (Conv2D)        (None, 29, 127, 32)       4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 63, 32)        0         
_________________________________________________________________
convolution2 (Conv2D)        (None, 14, 63, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 31, 32)         0         
__________

Next, we connect to AirSim

In [3]:
print('Connecting to AirSim...')
car_client = CarClient()
car_client.confirmConnection()
car_client.enableApiControl(False)
car_controls = CarControls()
print('Connected!')

Connecting to AirSim...
Waiting for connection: 
Connected!


Next, we define a helper function to obtain images from the simulator.

In [4]:
def get_image(car_client):
    image_response = car_client.simGetImages([ImageRequest(0, AirSimImageType.Scene, False, False)])[0]
    image1d = np.frombuffer(image_response.image_data_uint8, dtype=np.uint8)
    image_rgba = image1d.reshape(image_response.height, image_response.width, 4)

    return image_rgba[76:135,0:255,0:3].astype(float)

Finally, we start the main loop to drive the car. 

You should now see your car driving around using the model you just trained!

In [5]:
def append_to_ring_buffer(item, buffer, buffer_size):
    if (len(buffer) >= buffer_size):
        buffer = buffer[1:]
    buffer.append(item)
    return buffer

position_key = bytes('position', encoding='utf8')
x_val_key = bytes('x_val', encoding='utf8')
y_val_key = bytes('y_val', encoding='utf8')


x = []
y = []
state_buffer = []
state_buffer_len = 4

print('Running car for a few seconds...')
car_controls.steering = 0
car_controls.throttle = 1
car_controls.brake = 0
car_client.setCarControls(car_controls)
stop_run_time =datetime.datetime.now() + datetime.timedelta(seconds=2)
while(datetime.datetime.now() < stop_run_time):
    time.sleep(0.01)
    state_buffer = append_to_ring_buffer(get_image(car_client), state_buffer, state_buffer_len)

print('Running model')
while(True):
    data = np.column_stack((x, y))
    np.savetxt('car_point.txt', data, fmt= '%r')
    car_state =  car_client.getCarState()
    car_point = np.array([car_state.kinematics_true[position_key][x_val_key], car_state.kinematics_true[position_key][y_val_key], 0])
    x.append(car_point[0])
    y.append(car_point[1])
    
    state_buffer = append_to_ring_buffer(get_image(car_client), state_buffer, state_buffer_len)
    next_state, dummy = model.predict_state(state_buffer)
    next_control_signal = model.state_to_control_signals(next_state, car_client.getCarState())

    car_controls.steering = next_control_signal[0]
    car_controls.throttle = next_control_signal[1]
    car_controls.brake = next_control_signal[2]

    print('State = {0}, steering = {1}, throttle = {2}, brake = {3}'.format(next_state, car_controls.steering, car_controls.throttle, car_controls.brake))

    car_client.setCarControls(car_controls)

    time.sleep(0.1)

Running car for a few seconds...
Running model
State = 2, steering = 0, throttle = 1, brake = 0
State = 2, steering = 0, throttle = 1, brake = 0
State = 2, steering = 0, throttle = 1, brake = 0
State = 2, steering = 0, throttle = 1, brake = 0
State = 2, steering = 0, throttle = 1, brake = 0
State = 2, steering = 0, throttle = 1, brake = 0
State = 2, steering = 0, throttle = 1, brake = 0
State = 2, steering = 0, throttle = 1, brake = 0
State = 2, steering = 0, throttle = 1, brake = 0
State = 2, steering = 0, throttle = 1, brake = 0
State = 2, steering = 0, throttle = 1, brake = 0
State = 2, steering = 0, throttle = 1, brake = 0
State = 2, steering = 0, throttle = 1, brake = 0
State = 2, steering = 0, throttle = 1, brake = 0
State = 2, steering = 0, throttle = 0, brake = 1
State = 2, steering = 0, throttle = 0, brake = 1
State = 2, steering = 0, throttle = 0, brake = 1
State = 2, steering = 0, throttle = 0, brake = 1
State = 2, steering = 0, throttle = 0, brake = 1
State = 2, steering = 

State = 2, steering = 0, throttle = 0, brake = 1
State = 2, steering = 0, throttle = 0, brake = 1
State = 2, steering = 0, throttle = 0, brake = 1
State = 1, steering = -0.5, throttle = 0, brake = 1
State = 1, steering = -0.5, throttle = 0, brake = 1
State = 2, steering = 0, throttle = 0, brake = 1
State = 2, steering = 0, throttle = 0, brake = 1
State = 2, steering = 0, throttle = 0, brake = 1
State = 2, steering = 0, throttle = 0, brake = 1
State = 2, steering = 0, throttle = 0, brake = 1
State = 2, steering = 0, throttle = 0, brake = 1
State = 2, steering = 0, throttle = 0, brake = 1
State = 2, steering = 0, throttle = 0, brake = 1
State = 2, steering = 0, throttle = 0, brake = 1
State = 2, steering = 0, throttle = 0, brake = 1
State = 2, steering = 0, throttle = 0, brake = 1
State = 2, steering = 0, throttle = 0, brake = 1
State = 3, steering = 0.5, throttle = 0, brake = 1
State = 3, steering = 0.5, throttle = 0, brake = 1
State = 2, steering = 0, throttle = 0, brake = 1
State = 2,

KeyboardInterrupt: 